In [1]:
import ee
ee.Initialize()

In [2]:
dataset = ee.Image('USGS/SRTMGL1_003')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

In [3]:
# Extract pixel value in terms of slope
def extractPixelSlopeValue(x,slopeDB):
    p = ee.Geometry.Point(x)
    data =  slopeDB.select("slope").reduceRegion(ee.Reducer.first(),p,10).get("slope") 
    return ee.Number(data)

In [4]:
# Manipulate pixel value based on parametric formulae (Pressure in Pascal is calculated here)
def getPressureValue(pixelData):
    return ee.Number(101325).divide(pixelData.divide(ee.Number(8400)).exp())

In [5]:
import os
os.system("earthengine upload table --asset_id=users/nihalh55/MangaloreShapes Mangalore.zip")
table = ee.FeatureCollection("users/nihalh55/MangaloreShapes")

In [6]:
# Creating bounds
bounds = ee.Geometry.Polygon([[[74.856,12.9141],[77.5946,12.9716],[76.9654,11.0073],[75.7804,11.2588]]]);

In [22]:
polygon = slope.clip(bounds)

task_config = {
    'scale': 500,
    "description":'DEM-Data-Mangalore',
    'maxPixels': 1e12,
    'fileFormat': 'GeoTIFF',
    'folder': 'AM477',
    'region':table,
    'dimensions':720
    }

task = ee.batch.Export.image.toDrive(slope, **task_config)
task.start()

# proj = polygon.select([0]).projection()
# latlon = ee.Image.pixelLonLat().reproject(proj)

# coords =  latlon.select(['longitude', 'latitude']).reduceRegion({ "reducer": ee.Reducer.toList(), "geometry": bounds, "bestEffort": True, "scale": 369})
# lat = ee.List(coords.get('latitude'))
# lon = ee.List(coords.get('longitude'))
# point_list = lon.zip(lat)

# newImage = ee.Image.constant(ee.Number(101325)).divide(polygon.divide(ee.Image.constant(ee.Number(8400))).exp())

# Map.setCenter(75, 12.95, 8);
# Map.addLayer(polygon, {min: 0, max: 60}, 'bounded-area');
# Map.addLayer(newImage, {min: 100000, max: 101500}, 'pressure');

# ee.batch.Export.image.toDrive({"image":slope, 
#                       "description":'DEM-Data-Mangalore',
#                       "folder": 'AM477',
#                       "dimensions":720, 
#                       "region":table,
#                       "scale":500,
#                       "maxPixels":1e12,
#                       "fileFormat": 'GeoTIFF'});
